In [1]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from uuid import uuid4
from dotenv import load_dotenv
load_dotenv()

In [18]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv('AZURE_OPENAI_DEPLOYMENT_ID')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)

In [19]:
index_name = "ocbc-hr-gpt"  # change if desired

index = pc.Index(index_name)

In [20]:
llm = AzureChatOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
            api_version=AZURE_API_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0.0,
            verbose=True,
        )

embedding_llm = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment='embedding-ada-crayon',
            api_key=AZURE_OPENAI_KEY,
            api_version=AZURE_API_VERSION,
        )

In [21]:
vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)
retriever = vector_store.as_retriever()

In [22]:
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def caller(message):
    response = rag_chain.invoke(message)
    return response

In [24]:
retriever.invoke('Scope & Applicability ')

[Document(metadata={'page': 2.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content='3 \n \nOCBC Information Classification: Internal  Scope & Applicability  4 \nDefinition & Glossary  4 \nPolicy / Process  Error! Bookmark not def ined.  \n4.1 Policy Definition:  Error! Bookmark not defined.  \n4.2. Procedures  Error! Bookmark not defined.  \n4.3. Responsibility – HR department / Finance department  Error! Bookmark not \ndefined.  \nNon-compliance and consequences  1 \nSpecial Circumstances and Exceptions  Error! Bookmark not defined.'),
 Document(metadata={'page': 3.0, 'source': 'c:\\Users\\san\\Downloads\\Dummy - CB Policy.pdf'}, page_content="4 \n \nOCBC Information Classification: Internal   \n1. Objective  \n \nThe objective of the Compensation and Benefits Policy (The Policy) is aimed \ntowards building a strong framework of pay structure for the organization to \ncreate a  competitive work environment for its employees.  \nThe policy aims  to be transpar

In [25]:
caller('Scope & Applicability ')

"The scope and applicability of the Compensation and Benefits Policy is to define the principles of the organization's C&B structure and serve as a reference for employees and the HR department. The policy applies to all employees working with the organization. The policy aims to provide transparency and communicate compensation plans to employees."